In [1]:
import os
import cv2
import sys
import numpy as np
import pandas as pd
import mediapipe as mp
from tqdm.notebook import tqdm
from lib.phonology.temporal import TemporalPhonology
from lib.phonology.handshape import HandshapePhonology

: 

## Infer Phonological Feats

In [1]:
tqdm.pandas()

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

LEFT = HandshapePhonology('Left')
RIGHT = HandshapePhonology('Right')

PREFIX = 'data/corpus/'

DATA = []
SELECTED = ['LOCATION', 'ORIENTATION', 'FINGER_SELECTION', 'Path', 'Spacial', 'Temporal', 'Setting']
LEXICON = []

for video in tqdm(os.listdir(PREFIX)):
    FNAME = PREFIX + video
    
    LEXICON.append(video.split('.')[0])
    
    cam = cv2.VideoCapture(FNAME)
    LEFT = HandshapePhonology('Left')
    RIGHT = HandshapePhonology('Right')
    TEMPORAL = TemporalPhonology(SELECTED)
    
    with mp_holistic.Holistic(min_detection_confidence=0.1, min_tracking_confidence=0.1) as holistic:
        try:
            while cam.isOpened() and cam:
                ret, frame = cam.read()
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = holistic.process(image)
                
                RIGHT.getJointInfo(results.right_hand_landmarks, results.pose_landmarks, SELECTED)
                LEFT.getJointInfo(results.left_hand_landmarks, results.pose_landmarks, SELECTED)
                INFO = TEMPORAL.getTemporalInformation(LEFT, RIGHT)
        
        except cv2.error: pd.DataFrame.from_records(TEMPORAL.HISTORY).to_csv(f'data/vectors/{video.split(".")[0]}.csv', index=False)

  0%|          | 0/3395 [00:00<?, ?it/s]

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


KeyboardInterrupt: 

In [6]:

from sklearn.preprocessing import LabelEncoder

DATASET = []
for emb in tqdm(os.listdir('data/vectors/')[4:]):
    data_p = pd.read_csv('data/vectors/'+emb)
    data_p.replace(True, 1, inplace=True)
    data_p.replace(False, 0, inplace=True)

    categorical_cols = [i for i in data_p.columns if i.endswith('LABEL')]
    data_p = data_p.fillna(data_p.mean(numeric_only=True).round(1))
    #data_p = data_p.fillna(0)
    for col in categorical_cols:
        le = LabelEncoder()
        data_p[col] = le.fit_transform(data_p[col])
    data_p = data_p.fillna(data_p.mode(numeric_only=True).round(1))
    
    #r_cols = [i for i in data_p.columns if i.startswith('RIGHT')]
    #t_cols = [i for i in data_p.columns if (not i.startswith('LEFT')) and (not i.startswith('RIGHT'))]
    #
    #L= data_p[l_cols]
    #R= data_p[r_cols]
    #T= data_p[t_cols]
    #
    #print(L.shape)    
    #print(R.shape)    
    #print(T.shape)    
    #
    #print(np.dstack( (L.to_numpy(),R.to_numpy(),T.to_numpy()) ))
    
    
    
    DATASET.append(data_p.to_numpy())


  0%|          | 0/74 [00:00<?, ?it/s]

In [ ]:
!python3 -m pip install tensorflow
# Verify install:
!python3 -c "import tensorflow as tf; print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

In [11]:
import keras
from keras import layers

input_img = keras.Input(shape=(28, 28, 1))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Flatten()
encoded = layers.Dense(units=3136,activation='sigmoid')

# at this point the representation is (4, 4, 8) i.e. 128-dimensional
x = layers.Input(shape=(3136))(encoded)
x = layers.Dense(units=3136,activation='sigmoid')(x)
x = layers.Reshape((28,28,4))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

: 